In [1]:
import os

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

import sys
sys.path.append('..')
sys.path.append('../..')

In [2]:
import os
import time
import cv2
import png

import scipy.misc
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
%pylab inline

import dataset_main_test
from read_depth import depth_read
from python_flo import readFLO

sess = tf.InteractiveSession()

/home/bham/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/home/bham/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Populating the interactive namespace from numpy and matplotlib


In [3]:
ds_class = dataset_main_test.Dataset( )

test_path = '../../folder/eigen_test.txt'

_, _, test_list = ds_class.get_list( test_list_path=test_path )

print( 'Total test fol : ', len(test_list) )

Total test fol :  27


In [4]:
#KITTI_path = os.path.abspath('/media/chanho/datasets/KITTI_raw_data/')
KITTI_path = os.path.abspath('/mnt/sdb/datasets/KITTI_raw_data/')
INTP_path  = os.path.join( KITTI_path, 'depth_interpolation' )
GT_path = os.path.join(KITTI_path, 'eigen_test_gt_from_lidar')
RGB_path = os.path.join(KITTI_path, 'RGB')
OPT_path = os.path.join(KITTI_path, 'optflw_dis_inv')
# OPT_path = os.path.join('/mnt/sdd/PWC_flow')
#PRED_path  = os.path.abspath('/mnt/sdd/test_eigen/')

In [5]:
def scale(pred_path, gt_path):
    
    pred = depth_read(pred_path)
    gt   = depth_read(gt_path)
    
    mask = (gt != -1)
    gt[np.invert(mask)] = np.nan

    gt_median   = np.nanmedian(gt)
    pred_median = np.nanmedian(pred)
    
    pred *= gt_median / pred_median

    return pred
    
def bound(pred, min_dep=1e-3, max_dep=50):
    
    pred[pred<min_dep] = min_dep
    pred[pred>max_dep] = max_dep
    
    return pred

In [6]:
def idx(height, width):

    idx_x = [ [ i for i in range(width)] for j in range(height) ]
    idx_y = []

    for j in range(height):
        idx_y.append([ j for i in range(width) ])

    idx = np.zeros((height, width, 2))
    idx[:, :, 0] = idx_x
    idx[:, :, 1] = idx_y

    idx_tensor = tf.constant(idx, dtype=tf.float32)

    return idx_tensor

def warp(pst, back_flw):

    h, w, _ = back_flw.get_shape().as_list()
    index = idx(h, w)

    a_1 = tf.zeros_like(pst)
    a_2 = tf.zeros_like(pst)
    a_3 = tf.zeros_like(pst)
    a_4 = tf.zeros_like(pst)

    x_n = tf.clip_by_value(index[:,:,0] + back_flw[:,:,0], 0, w-1)
    y_n = tf.clip_by_value(index[:,:,1] + back_flw[:,:,1], 0, h-1)

    x = tf.clip_by_value(tf.floor(index[:,:,0] + back_flw[:,:,0]), 0, w-1)
    y = tf.clip_by_value(tf.floor(index[:,:,1] + back_flw[:,:,1]), 0, h-1)

    x_1 = tf.clip_by_value(x+1, 0, w-1)
    y_1 = tf.clip_by_value(y+1, 0, h-1)

    a_1 = tf.multiply(tf.expand_dims(tf.multiply(1.0-tf.abs(x_n - x), 1.0-tf.abs(y_n - y)), axis=2),
                      tf.gather_nd(pst, tf.to_int32(tf.stack([y,x],axis=2))))
    a_2 = tf.multiply(tf.expand_dims(tf.multiply(1.0-tf.abs(x_n - (x+1)), 1.0-tf.abs(y_n - y)), axis=2),
                      tf.gather_nd(pst, tf.to_int32(tf.stack([y,x_1],axis=2))))
    a_3 = tf.multiply(tf.expand_dims(tf.multiply(1.0-tf.abs(x_n - x), 1.0-tf.abs(y_n - (y+1))), axis=2),
                      tf.gather_nd(pst, tf.to_int32(tf.stack([y_1,x],axis=2))))
    a_4 = tf.multiply(tf.expand_dims(tf.multiply(1.0-tf.abs(x_n - (x+1)), 1.0-tf.abs(y_n - (y+1))), axis=2),
                      tf.gather_nd(pst, tf.to_int32(tf.stack([y_1,x_1],axis=2))))

    return a_1 + a_2 + a_3 + a_4

def pred_warp(prev_pred, backflw):
    
    return tf.map_fn(lambda inputs : warp(inputs[0],inputs[1]), elems=[prev_pred, backflw], dtype=tf.float32)

def no_mask(appr_X, p_appr_X, backflw, alpha=0.5):

    wrpd_p_appr_X = tf.map_fn(lambda inputs : warp(inputs[0],inputs[1]), elems=[p_appr_X, backflw], dtype=tf.float32)

    diff = tf.reduce_mean(tf.abs(appr_X - wrpd_p_appr_X), 3) 
    msk  = tf.exp((-1) * alpha * diff)

    return msk
    
def readFLO(file_path):
    with open(file_path, 'rb') as f:
        magic = np.fromfile(f, np.float32, count=1)
        
        if 202021.25 != magic:
            print('Magic number incorrect. Invalid .flo file')
            return
        
        else:
            w = np.fromfile(f, np.int32, count=1)[0]
            h = np.fromfile(f, np.int32, count=1)[0]
            #print('Reading %d x %d flo file' % (h, w))
            data = np.fromfile(f, np.float32, count=2*w*h)
            # Reshape data into 3D array (columns, rows, bands)
            data2D = np.resize(data, (h, w, 2))
            return data2D

In [7]:
def first_order(PRED_path, SAVE_path, ours=False):

    total_error = 0.
    cnt = 0
    
    height = 376
    width  = 1242
    
    p_appr_X = tf.placeholder(tf.float32, [1, height, width, 3])
    appr_X   = tf.placeholder(tf.float32, [1, height, width, 3])
    tmpr_X   = tf.placeholder(tf.float32, [1, height, width, 2])
    p_dep    = tf.placeholder(tf.float32, [1, height, width])

    mask = tf.squeeze(no_mask(appr_X, p_appr_X, tmpr_X, alpha=0.5)) > 0.05
    wrpd_prev_pred = tf.squeeze(pred_warp(tf.expand_dims(p_dep, 3), tmpr_X))

    for fol in range(len(test_list)):
    #for fol in range(1):

        video_split = len(next(os.walk('/mnt/sdb/datasets/KITTI_raw_data/RGB/%s%s/' % (test_list[fol], 'image_02/data')))[2])

        for frame_num in range(2, video_split-1):
        #for frame_num in range(2, 3):
            
            #< ----------------------- Data Path ----------------------- >#
            full = test_list[fol].split()[0]
            split = test_list[fol].split()[0].split('/')

            if ours:
                
                prev_pred_path = os.path.join(PRED_path, full, '%010d.png'%(frame_num-1))
                pred_path      = os.path.join(PRED_path, full, '%010d.png'%frame_num)
            
            else:
                
                prev_pred_path = os.path.join(PRED_path, full, 'image_02/data', '%010d.png'%(frame_num-1))        
                pred_path      = os.path.join(PRED_path, full, 'image_02/data', '%010d.png'%frame_num)
                
            prev_gt_path  = os.path.join(GT_path, full, 'image_02', '%010d.png'%(frame_num-1))
            gt_path       = os.path.join(GT_path, full, 'image_02', '%010d.png'%frame_num)
            prev_rgb_path = os.path.join(RGB_path, full, 'image_02', 'data', '%010d.png'%(frame_num-1))
            rgb_path      = os.path.join(RGB_path, full, 'image_02', 'data', '%010d.png'%frame_num)
            flw_path      = os.path.join(OPT_path, full, 'image_02', 'data', '%010d.flo'%frame_num)

            #< ----------------------- Data Load  ----------------------- >#

            prev_pred = scale(prev_pred_path, prev_gt_path)
            pred = scale(pred_path, gt_path)
            prev_rgb = scipy.misc.imread(prev_rgb_path)
            rgb = scipy.misc.imread(rgb_path)
            flw = readFLO(flw_path)

            #rgb_h, rgb_w, _  = np.shape(rgb)
            prev_pred = cv2.resize(np.squeeze(prev_pred), (width, height), interpolation=cv2.INTER_AREA)
            pred = cv2.resize(np.squeeze(pred), (width, height), interpolation=cv2.INTER_AREA)
            prev_rgb = cv2.resize(np.squeeze(prev_rgb), (width, height), interpolation=cv2.INTER_AREA)
            rgb = cv2.resize(np.squeeze(rgb), (width, height), interpolation=cv2.INTER_AREA)
            flw = cv2.resize(np.squeeze(flw), (width, height), interpolation=cv2.INTER_AREA)
            
            M = mask.eval(feed_dict={p_appr_X: [prev_rgb], appr_X: [rgb], tmpr_X: [flw]})
            Wrpd = wrpd_prev_pred.eval(feed_dict={p_dep: [prev_pred], tmpr_X: [flw]})

            pred_diff_1 = M * np.abs(Wrpd - pred)
            masked_mean_error = np.sum(pred_diff_1) / np.sum(M)

            total_error += masked_mean_error
            cnt += 1

            if not os.path.exists(os.path.join(SAVE_path, split[0])):
                os.makedirs(os.path.join(SAVE_path, split[0]))
            if not os.path.exists(os.path.join(SAVE_path, split[0], split[1])):
                os.makedirs(os.path.join(SAVE_path, split[0], split[1]))

            with open(os.path.join(SAVE_path, split[0], split[1], 'log.txt'), 'a') as f:
                f.write('%.5f\n'%(masked_mean_error))
                
    return total_error / cnt


In [8]:
def gt_first_order(PRED_path, SAVE_path):

    total_error = 0.
    cnt = 0
    
    height = 376
    width  = 1242
    
    p_appr_X = tf.placeholder(tf.float32, [1, height, width, 3])
    appr_X   = tf.placeholder(tf.float32, [1, height, width, 3])
    tmpr_X   = tf.placeholder(tf.float32, [1, height, width, 2])
    p_dep    = tf.placeholder(tf.float32, [1, height, width])

    mask = tf.squeeze(no_mask(appr_X, p_appr_X, tmpr_X, alpha=0.5)) > 0.05
    wrpd_prev_pred = tf.squeeze(pred_warp(tf.expand_dims(p_dep, 3), tmpr_X))

    for fol in range(len(test_list)):
    #for fol in range(1):

        video_split = len(next(os.walk('/mnt/sdb/datasets/KITTI_raw_data/RGB/%s%s/' % (test_list[fol], 'image_02/data')))[2])

        for frame_num in range(2, video_split-1):
        #for frame_num in range(2, 3):
            
            #< ----------------------- Data Path ----------------------- >#
            full = test_list[fol].split()[0]
            split = test_list[fol].split()[0].split('/')
                
            prev_pred_path = os.path.join(PRED_path, full, 'proj_depth/groundtruth/image_02', '%010d.png'%(frame_num-1))        
            pred_path      = os.path.join(PRED_path, full, 'proj_depth/groundtruth/image_02', '%010d.png'%frame_num)
            prev_rgb_path = os.path.join(RGB_path, full, 'image_02', 'data', '%010d.png'%(frame_num-1))
            rgb_path      = os.path.join(RGB_path, full, 'image_02', 'data', '%010d.png'%frame_num)
            flw_path      = os.path.join(OPT_path, full, 'image_02', 'data', '%010d.flo'%frame_num)

            #< ----------------------- Data Load  ----------------------- >#

            prev_pred = depth_read(prev_pred_path)
            pred = depth_read(pred_path)
            prev_rgb = scipy.misc.imread(prev_rgb_path)
            rgb = scipy.misc.imread(rgb_path)
            flw = readFLO(flw_path)

            #rgb_h, rgb_w, _  = np.shape(rgb)
            prev_pred = cv2.resize(np.squeeze(prev_pred), (width, height), interpolation=cv2.INTER_AREA)
            pred = cv2.resize(np.squeeze(pred), (width, height), interpolation=cv2.INTER_AREA)
            prev_rgb = cv2.resize(np.squeeze(prev_rgb), (width, height), interpolation=cv2.INTER_AREA)
            rgb = cv2.resize(np.squeeze(rgb), (width, height), interpolation=cv2.INTER_AREA)
            flw = cv2.resize(np.squeeze(flw), (width, height), interpolation=cv2.INTER_AREA)
            
            garg_crop = np.array([0.5*height, 0.99189189*height, 0.03594771*width, 0.96405229*width]).astype(np.int32)
            crop_mask = np.zeros((height, width))
            crop_mask[garg_crop[0]:garg_crop[1],garg_crop[2]:garg_crop[3]] = 1
            M = mask.eval(feed_dict={p_appr_X: [prev_rgb], appr_X: [rgb], tmpr_X: [flw]})

            MSK  = np.logical_and(crop_mask, M)
            Wrpd = wrpd_prev_pred.eval(feed_dict={p_dep: [prev_pred], tmpr_X: [flw]})

            pred_diff_1 = MSK * np.abs(Wrpd - pred)
            masked_mean_error = np.sum(pred_diff_1) / np.sum(MSK)

            total_error += masked_mean_error
            cnt += 1

            if not os.path.exists(os.path.join(SAVE_path, split[0])):
                os.makedirs(os.path.join(SAVE_path, split[0]))
            if not os.path.exists(os.path.join(SAVE_path, split[0], split[1])):
                os.makedirs(os.path.join(SAVE_path, split[0], split[1]))

            with open(os.path.join(SAVE_path, split[0], split[1], 'log.txt'), 'a') as f:
                f.write('%.5f\n'%(masked_mean_error))
                
    return total_error / cnt


In [9]:
save_path = './temp/final1_DIS/'

In [ ]:
SAVE_path = os.path.abspath(save_path + 'GT')
error1 = gt_first_order(INTP_path, SAVE_path)
print(error1)

with open(os.path.join(save_path, 'total.txt'), 'a') as f:
    f.write('%30s %.5f\n'%('GT', error1))

In [ ]:
PRED_path = os.path.abspath('/mnt/sdd/flowgru_other_methods/SfMLearner-master_Zhou/depth/')

SAVE_path = os.path.abspath(save_path + 'Zhou')
error1 = first_order(PRED_path, SAVE_path)
print(error1)

with open(os.path.join(save_path, 'total.txt'), 'a') as f:
    f.write('%30s %.5f\n'%('Zhou', error1))

In [ ]:
PRED_path = os.path.abspath('/mnt/sdd/flowgru_other_methods/LKVOLearner-master_Wang/src/depth/')

SAVE_path = os.path.abspath(save_path + 'Wang')
error1 = first_order(PRED_path, SAVE_path)
print(error1)

with open(os.path.join(save_path, 'total.txt'), 'a') as f:
    f.write('%30s %.5f\n'%('Wang', error1))

In [ ]:
PRED_path = os.path.abspath('/mnt/sdd/flowgru_other_methods/GeoNet-master/depth/')

SAVE_path = os.path.abspath(save_path + 'GeoNet')
error1 = first_order(PRED_path, SAVE_path)
print(error1)

with open(os.path.join(save_path, 'total.txt'), 'a') as f:
    f.write('%30s %.5f\n'%('GeoNet', error1))

In [ ]:
PRED_path = os.path.abspath('/home/bham/HyunChan/Godard-monodepth-master/depth/')

SAVE_path = os.path.abspath(save_path + 'Godard')
error1 = first_order(PRED_path, SAVE_path)
print(error1)

with open(os.path.join(save_path, 'total.txt'), 'a') as f:
    f.write('%30s %.5f\n'%('Godard', error1))

In [ ]:
PRED_path = os.path.abspath('/mnt/sdd/flowgru_other_methods/Garg/depth_80/')

SAVE_path = os.path.abspath(save_path + 'Garg')
error1 = first_order(PRED_path, SAVE_path)
print(error1)

with open(os.path.join(save_path, 'total.txt'), 'a') as f:
    f.write('%30s %.5f\n'%('Garg', error1))

In [ ]:
PRED_path = os.path.abspath('/mnt/sdd/flowgru_other_methods/semodepth-master_Kuznietsov/inference/depth/')

SAVE_path = os.path.abspath(save_path + 'Kuznietsov')
error1 = first_order(PRED_path, SAVE_path)
print(error1)

with open(os.path.join(save_path, 'total.txt'), 'a') as f:
    f.write('%30s %.5f\n'%('Kuznietsov', error1))

In [ ]:
PRED_path = os.path.abspath('/mnt/sdd/flowgru_other_methods/DORN-master_Fu/depth/')

SAVE_path = os.path.abspath(save_path + 'Fu')
error1 = first_order(PRED_path, SAVE_path)
print(error1)

with open(os.path.join(save_path, 'total.txt'), 'a') as f:
    f.write('%30s %.5f\n'%('Fu', error1))

In [ ]:
PRED_path = os.path.abspath('/mnt/sdd/test_eigen/main_lrsch_smth_multiGPU_193')

SAVE_path = os.path.abspath(save_path + 'main_lrsch_smth_multiGPU_193')
error1 = first_order(PRED_path, SAVE_path, True)
print(error1)

with open(os.path.join(save_path, 'total.txt'), 'a') as f:
    f.write('%30s %.5f\n'%('main_lrsch_smth_multiGPU_193', error1))

In [ ]:
PRED_path = os.path.abspath('/mnt/sdd/test_eigen/base_flowgru_multiGPU2_180')

SAVE_path = os.path.abspath(save_path + 'base_flowgru_multiGPU_180')
error1 = first_order(PRED_path, SAVE_path, True)
print(error1)

with open(os.path.join(save_path, 'total.txt'), 'a') as f:
    f.write('%30s %.5f\n'%('base_flowgru_multiGPU_180', error1))

In [ ]:
PRED_path = os.path.abspath('/mnt/sdd/test_eigen/base_flowgru_multiGPU2_CS_KITTI_epoch085')

SAVE_path = os.path.abspath(save_path + 'base_flowgru_multiGPU_CS_KITTI_epoch085')
error1 = first_order(PRED_path, SAVE_path, True)
print(error1)

with open(os.path.join(save_path, 'total.txt'), 'a') as f:
    f.write('%30s %.5f\n'%('base_flowgru_multiGPU_CS_KITTI_epoch085', error1))

In [21]:
PRED_path = os.path.abspath('/mnt/sdd/test_eigen/main_lrsch_smth_multiGPU_100')

SAVE_path = os.path.abspath(save_path + 'main_lrsch_smth_multiGPU_100')
error1 = first_order(PRED_path, SAVE_path, True)
print(error1)

1.042013934292797


In [22]:
PRED_path = os.path.abspath('/mnt/sdd/test_eigen/base_flowgru_multiGPU2_wofrn_096')

SAVE_path = os.path.abspath(save_path + 'base_flowgru_multiGPU2_wofrn_096')
error1 = first_order(PRED_path, SAVE_path, True)
print(error1)

1.0273154337679231


In [23]:
PRED_path = os.path.abspath('/mnt/sdd/test_eigen/base_flowgru_multiGPU2_100')

SAVE_path = os.path.abspath(save_path + 'base_flowgru_multiGPU_100')
error1 = first_order(PRED_path, SAVE_path, True)
print(error1)

1.0090761705663338
